# Optimization of liquid transfer paramters of viscous liquids guided by Baysesian Optimization

## 1. Introduction

This jupyter notebook contains the information required to perform Multi-Bayesian optimization (MOBO) of liquid handling parameters of pipetting robots according to the protocol described in **publication add link** . The notebook is divided in the following sections

1. **Imports**: This section contains the relevant packages required to perform MOBO

2. **BO Liquid Tansfer class**: This section defines the BO_LiqTransfer class that contains the functions required to implement the MOBO through semi-automataed and fully automated methods. The semi-autmated method is prefered when the method to control the liquid transfering robot cannot be implemented with external libraries such as Torch. the fully-automated implementation in this notebook is defined to control a rLine1000 Sartorious pipette coupled to a M1 Dobot Scara robotic arm controlled by control-lab-ly python package.

3. **Semi-automated implemntation**: This section contains the code to obtain  suggestions for liquid handling parameters where the robotic platform performing the transfers is controlled in a separte script. This is the method that was used in **publication** to optimize the liquid transfer paramters of a Opentrons OT2 robot.

4. **Fully-automated implemetnation**: This section contains the code to obtain  suggestions for liquid handling parameters where the robotic platform performing the transfers is a rLine1000 Sartorious pipette coupled to a M1 Dobot Scara robotic arm controlled by control-lab-ly python package. This is the method that was used in **publication** to optimize the liquid transfer paramters of a rLine1000 pipette in the fully automated optimization experiments.

## 1. Imports

In [ ]:
# basic dependencies

# basic dependencies

import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

import pandas as pd
import time


## To process mass data and fit sigmid curves in automated initialization experiments
from scipy import signal
from scipy.optimize import curve_fit

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly

torch.set_printoptions(precision=3)

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement

# utilities
from botorch.optim.optimize import optimize_acqf
from botorch.sampling import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

from gpytorch.constraints import GreaterThan
from torch.optim import SGD
from gpytorch.mlls import ExactMarginalLogLikelihood

import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)



## 2. Bayesian optimization liquid transfer class

In [ ]:
class BO_LiqTransfer:

    def __init__(self, liquid_name, density):
        self.liquid_name = liquid_name
        self.density = density
        self._data = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate', 'delay_aspirate', 'delay_dispense','iteration','liquid_level','density', 'time', 'm', '%error','time_asp_1000','acq_value'])
        self.features = ['aspiration_rate','dispense_rate']
        self.objectives = ['%error','time_asp_1000']
        self.bmax = 1.25
        self.bmin = 0.1
        self._first_approximation = None
        self._latest_suggestion = None
        self._latest_volume = None
        self._latest_acq_value = None
        self.mean_volumes = [1000,500,300]
        self.platform = None
    
   
    def cleanTip(self, well, repetitions =2 ):
        
        self.platform.mover.safeMoveTo(well.top)
        
        for i in range(repetitions):
            self.platform.liquid.blowout(home=False) 
            time.sleep(5)
            self.platform.liquid.home()
            self.platform.setup.touchTip(well)
            time.sleep(5)

            self.platform.liquid.blowout(home=False) 
            time.sleep(5)
            self.platform.liquid.home()
            self.platform.setup.touchTip(well)
            time.sleep(5)

            self.platform.liquid.blowout(home=False) 
            time.sleep(5)
            self.platform.liquid.home()
            self.platform.setup.touchTip(well)
            time.sleep(5)

    
    def gravimetric_transfer(self, volume, liquid_level, source_well,balance_well, aspiration_rate, dispense_rate):
        #liquid transfer
        #transfer start
        start = time.time() 

        #aspirate step
        self.platform.mover.safeMoveTo(source_well.from_bottom((0,0,liquid_level-5))) 
        self.platform.liquid.aspirate(volume, speed=aspiration_rate)
        time.sleep(10)

        self.platform.setup.touchTip(source_well) 

        #dispense step
        self.platform.mover.safeMoveTo(balance_well.from_top((0,0,-5))) 
        self.platform.balance.tare() 
        self.platform.balance.clearCache() 
        self.platform.balance.toggleRecord(True) 
        time.sleep(5)
        self.platform.liquid.dispense(volume,speed=dispense_rate)
        time.sleep(10)

        #transfer termination
        finish = time.time() 
        time_m = finish - start

        self.platform.mover.safeMoveTo(source_well.top) 
        time.sleep(5)
        self.platform.balance.toggleRecord(False) 

        #do blowout
        
        self.cleanTip(source_well)

        #record transfer values 
        #calculating mass error functions
        m = (self.platform.balance.buffer_df.iloc[-10:,-1].mean()-self.platform.balance.buffer_df.iloc[:10,-1].mean())/1000 
        error = (m-self.density*volume/1000)/(self.density/1000*volume)*100
        
        #change liquid levels
        liquid_level = liquid_level - 2*m/self.density   
        
        #making new dataframe + filling it in
        df = pd.DataFrame(columns=self._data.columns)            
        
        df = pd.concat([df,pd.DataFrame({
            "liquid": self.liquid_name,
            'pipette': 'rLine1000',
            "volume": volume,
            "aspiration_rate": aspiration_rate,
            "dispense_rate": dispense_rate, 
            "delay_aspirate" : 10,  
            "delay_dispense" : 10,
            'iteration': 'NaN',
            "liquid_level" : liquid_level,
            "density" : self.density,
            "time" : time_m,
            "m": m,
            "%error": error,
            "time_asp_1000" : 'NaN',
            "acq_value": self._latest_acq_value
            },index=[0])],ignore_index=True)
        
        return liquid_level, df


    def obtainAproximateRate(self, initial_liquid_level_balance,balance_well,speed=265, file_name=None):

        liquid_level = initial_liquid_level_balance
        
        if self.platform.liquid.isTipOn()== False:
            self.platform.setup.attachTip()
        
        self.platform.mover.safeMoveTo(balance_well.from_bottom((0,0,liquid_level-5)),descent_speed_fraction=0.25)
        #Starting balance measurement
        time.sleep(5)
        self.platform.balance.zero(wait=5)
        self.platform.balance.clearCache()
        self.platform.balance.toggleRecord(on=True)
        time.sleep(15)

        self.platform.liquid.aspirate(1000, speed=speed)

        #Switching the balance off after change in mass is less than 0.05
        while True:
            data = self.platform.balance.buffer_df
            data['Mass_smooth']= signal.savgol_filter(data['Mass'],91,1)
            data['Mass_derivative_smooth']=data['Mass_smooth'].diff()
            condition=data['Mass_derivative_smooth'].rolling(30).mean().iloc[-1]
            if condition>-0.05:
                break
        print('loop stopped')
        self.platform.balance.toggleRecord(on=False)

        self.platform.mover.moveTo(balance_well.from_top((0,0,-5)))

        def sigmoid(x, K ,x0, B,v,A):
            y = (K-A) / (1 + np.exp(B*(x-x0)))**(1/v) + A
            return y


        #using data from balance buffer_df above, calculate time in seconds and mass derivatives
        data['ts'] = data['Time'].astype('datetime64[ns]').values.astype('float') / 10 ** 9
        data['ts']= data['ts']-data['ts'][0]
        data_fit = data.where(data['ts']>10).dropna()
        data_fit['Mass']=data_fit['Mass']-data_fit['Mass'].iloc[0]
        data_fit['Mass_smooth'] = data_fit['Mass_smooth']-data_fit['Mass_smooth'].iloc[0]

        p0 = [min(data_fit['Mass']), np.median(data_fit['ts']),1,1,max(data_fit['Mass'])+30]
        
        popt, pcov = curve_fit(sigmoid, data_fit['ts'], data_fit['Mass'],p0)

        mass_sigmoid = sigmoid(data_fit['ts'],popt[0],popt[1],popt[2],popt[3],popt[4])

        data_fit.loc[data_fit.index[0]:,'Mass_sigmoid'] = mass_sigmoid

        flow_rate = mass_sigmoid.diff()/data_fit.loc[data_fit.index[0]:,'ts'].diff()

        data_fit.loc[data_fit.index[0]:,'Flow_rate']=flow_rate

        flow_rate = mass_sigmoid.diff()/data_fit.loc[data_fit.index[0]:,'ts'].diff()

        flow_rate_max = flow_rate.min()

        flow_rate_98 = data_fit.where(data_fit['Flow_rate']<(0.05*flow_rate_max)).dropna()

        time_start, time_final = flow_rate_98.iloc[0].loc['ts'],flow_rate_98.iloc[-1].loc['ts']

        initial_flow_rate_aspirate = 1000/(time_final-time_start)
        
        self._first_approximation = initial_flow_rate_aspirate 
        
        #switching balance off and saving csv
        if file_name != None:
            data_fit.to_csv(file_name, index=False)

        self.platform.liquid.dispense(1000,speed= self._first_approximation)


    def exploreBoundaries(self, initial_liquid_level_source,source_well,balance_well):
        self.platform.mover.setSpeed(50)
        self.platform.mover.setHandedness(False)

        if type(self._data) == None:
            df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate', 'delay_aspirate', 'delay_dispense','iteration','liquid_level','density', 'time', 'm', '%error','time_asp_1000','acq_value'])
            df = df.astype({'liquid':str,'pipette':str})
            self.set_data(df)
        
        liquid_level = initial_liquid_level_source

        #Check if new tip is required
        if self.platform.liquid.isTipOn()== False:
            self.platform.setup.attachTip()

        volumes_list = self.mean_volumes
        
        #NOT TO BE CHANGED
        counter = 1
        iterations = 5
        #while loop
        while counter <= iterations:
            #hardcoding aspirate and dispense rates:
            if counter == 1:
                aspiration_rate = self._first_approximation
                dispense_rate = self._first_approximation
            if counter == 2:
                aspiration_rate = self._first_approximation*self.bmax
                dispense_rate = self._first_approximation*self.bmax
            if counter == 3:
                aspiration_rate = self._first_approximation*self.bmax
                dispense_rate = self._first_approximation*self.bmin
            if counter == 4:
                aspiration_rate = self._first_approximation*self.bmin
                dispense_rate = self._first_approximation*self.bmax
            if counter == 5:
                aspiration_rate = self._first_approximation*self.bmin
                dispense_rate = self._first_approximation*self.bmin


            #for loop
            for volume in volumes_list:
                #liquid transfer
                liquid_level,df = self.gravimetric_transfer(volume,liquid_level,source_well,balance_well,aspiration_rate,dispense_rate)
                
                m=df.m.iloc[-1]

                self.set_data(pd.concat([self._data,df]).reset_index(drop=True))
                #printing checks
                print("LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/self.density) + "   ITERATION: " + str(counter) + ", " + "VOLUME: " + str(volume))    

                #liquid level checks
                if (1.2*m/self.density > 1.2) or (1.2*m/self.density < 0):
                    break
                if (liquid_level > initial_liquid_level_source) or (liquid_level < 6):
                    break

            counter += 1


    def optimizeParameters(self, initial_liquid_level_source,source_well,balance_well,iterations=5,file_name=None):
        self.platform.mover.setSpeed(50)
        self.platform.mover.setHandedness(False)

        liquid_level = initial_liquid_level_source

        #Check if new tip is required
        if self.platform.liquid.isTipOn()== False:
            self.platform.setup.attachTip()

        volumes_list = self.mean_volumes
        
        #NOT TO BE CHANGED
        counter = 1
       
        #while loop
        while counter <= iterations:
            #getting botorch suggestions + implementing it in liquids_dict
            self.optimized_suggestions()
            aspiration_rate = self._latest_suggestion['aspiration_rate']
            dispense_rate = self._latest_suggestion['dispense_rate']
            #for loop
            for volume in volumes_list:
                #liquid transfer
                liquid_level,df = self.gravimetric_transfer(volume,liquid_level,source_well,balance_well,aspiration_rate,dispense_rate)
                
                m=df.m.iloc[-1]

                self.set_data(pd.concat([self._data,df]).reset_index(drop=True))
                #printing checks
                print("LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/self.density) + "   ITERATION: " + str(counter) + ", " + "VOLUME: " + str(volume))    


                
                #printing checks
                print("LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/self.density) + "   ITERATION: " + str(counter) + ", " + "VOLUME: " + str(volume))    

                #liquid level checks
                if (1.2*m/self.density > 1.2) or (1.2*m/self.density < 0):
                    break
                if (liquid_level > initial_liquid_level_source) or (liquid_level < 6):
                    break
            
            counter += 1
        if file_name != None:
            self._data.to_csv(file_name, index=False)


    @staticmethod
    def calibration_summary(df):

        if 'volume_transfered' and 'volume_error' and 'time_asp_1000' not in df.columns:
            df['volume_transfered'] = (df['m']/df['density'])*1000
            df['volume_error'] = df['volume_transfered'] - df['volume']
            df['time_asp_1000']=1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']             

        df_summary_all = pd.DataFrame()

        for volume in df['volume'].unique():
            df_experiment_v = df.where(df['volume'] == volume).dropna(how='all')
            df_summary = pd.DataFrame(columns = (f'Mean transfer volume for {volume} µL [µL]', f'Mean transfer volume error of {volume} µL [µL]', f'Mean relative error for transfer of {volume} µL [%]', f'Standard deviation for transfer of {volume} µL [µL]', f'Relative standard deviation for transfer of {volume} µL [%]') )
            data = [df_experiment_v['volume_transfered'].mean(), df_experiment_v['volume_error'].mean(), df_experiment_v['%error'].mean(), df_experiment_v['volume_transfered'].std(), (df_experiment_v['volume_transfered'].std() / df_experiment_v['volume_transfered'].mean() * 100)]
            df_summary.loc[df['liquid'].iloc[0]] = data
            df_summary_all = pd.concat([df_summary_all, df_summary], axis = 1)
        return df_summary_all   



    def calibrateParameters(self, initial_liquid_level_source,source_well,balance_well,iterations=10, file_name=None):

        self.platform.mover.setSpeed(50)
        self.platform.mover.setHandedness(False)

        liquid_level = initial_liquid_level_source

        # Check if new tip is required
        if self.platform.liquid.isTipOn()== False:
            self.platform.setup.attachTip()

        volumes_list = self.mean_volumes
        
        #NOT TO BE CHANGED
     
        
        mean_average_data = self._data.where(self._data.volume == 'mean'+str(self.mean_volumes))
        mean_average_data = mean_average_data.where(mean_average_data.iteration>5).dropna()
        best_parameter_index = mean_average_data[mean_average_data['%error']==mean_average_data['%error'].max()].index

        aspiration_rate = self._data.loc[best_parameter_index,'aspiration_rate'].values[0]
        dispense_rate = self._data.loc[best_parameter_index,'dispense_rate'].values[0]
        
        calibration_df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate', 'delay_aspirate', 'delay_dispense','liquid_level','density', 'm', '%error'])
        
        #for loop
            
        for volume in volumes_list:
            counter = 1
            # #while loop
            while counter <= iterations:
             
                #liquid transfer
                liquid_level,df = self.gravimetric_transfer(volume,liquid_level,source_well,balance_well,aspiration_rate,dispense_rate)
                
                calibration_df = pd.concat([calibration_df,df[calibration_df.columns]]).reset_index(drop=True)

                m=df.m.iloc[-1]

                #printing checks
                print("Mass: "+str(m)+"LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/self.density) + "   ITERATION: " + str(counter) + ", " + "VOLUME: " + str(volume))    

                #liquid level checks
                if (1.2*m/self.density > 1.2) or (1.2*m/self.density < 0):
                    break
                if (liquid_level > initial_liquid_level_source) or (liquid_level < 6):
                    break

                counter += 1
            #liquid level checks
            if (1.2*m/self.density > 1.2) or (1.2*m/self.density < 0):
                break
            if (liquid_level > initial_liquid_level_source) or (liquid_level < 6): 
                break
        
        calibration_df['volume_transfered'] = calibration_df['m']/calibration_df['density']*1000
        calibration_df['volume_error'] = calibration_df['volume_transfered'] - calibration_df['volume']
        calibration_df['time_asp_1000'] = 1000/calibration_df['aspiration_rate'] + 1000/calibration_df['dispense_rate'] + calibration_df['delay_aspirate'] + calibration_df['delay_dispense']       
        
        calibration_summary_df= self.calibration_summary(calibration_df)

        if file_name != None:
            calibration_df.to_csv(file_name, index=False)
            calibration_summary_df.to_csv(file_name[:-4]+'_summary.csv')
        




    #BO relevant functions                            

    def xy_split(self):
        df_train = self._data.where(self._data['volume']=='mean'+str(self.mean_volumes)).dropna(how='all')
        x_train = df_train[self.features]
        y_train = df_train[self.objectives]
        return x_train,y_train


    def set_bounds(self, x_train):
        return torch.vstack([x_train[0]*self.bmin, x_train[0]*self.bmax])


    def fit_surrogate(self):
        x_train, y_train = self.xy_split()
        x_train = torch.tensor(x_train.to_numpy(dtype=float), **tkwargs)
        y_train = torch.tensor(y_train.to_numpy(dtype=float), **tkwargs)
        y_train[:,0] = -torch.absolute(y_train[:,0])
        y_train[:,1] = -torch.absolute(y_train[:,1])

        problem_bounds = self.set_bounds(x_train)
        time_upper = 1000/problem_bounds[0][0] +1000/problem_bounds[0][1] + 10
        error_upper = y_train[:,0].abs().min()*1.25
        ref_point = torch.tensor([-error_upper,-time_upper], **tkwargs)

        train_x_gp = normalize(x_train, problem_bounds)
        models = []
        for i in range(y_train.shape[-1]):
            models.append(SingleTaskGP(train_x_gp, y_train[..., i : i + 1], outcome_transform=Standardize(m=1)))
        model1 = ModelListGP(*models)
        mll1 = SumMarginalLogLikelihood(model1.likelihood, model1)

        fit_gpytorch_model(mll1)
    
        return model1, ref_point, train_x_gp, problem_bounds
    

    def optimized_suggestions(self, random_state= 42):
        if random_state != None:
            torch.manual_seed(random_state) 
        standard_bounds = torch.zeros(2, len(self.features), **tkwargs)
        standard_bounds[1] = 1
        model1, ref_point, train_x_gp, problem_bounds = self.fit_surrogate()
        acq_func1 = qNoisyExpectedHypervolumeImprovement(
        model=model1,
        ref_point=ref_point, # for computing HV, must flip for BoTorch
        X_baseline=train_x_gp, # feed total list of train_x for this current iteration
        sampler=SobolQMCNormalSampler(sample_shape=512),  # determines how candidates are randomly proposed before selection
        objective=IdentityMCMultiOutputObjective(outcomes=np.arange(len(self.objectives)).tolist()), # optimize first n_obj col 
        prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on
        sobol1 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol2 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol_all = torch.vstack([sobol1, sobol2])
            
        acq_value_list = []
        for i in range(0, sobol_all.shape[0]):
            with torch.no_grad():
                acq_value = acq_func1(sobol_all[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())
                
        # filter the best 12 QMC candidates first
        sorted_x = sobol_all.cpu().numpy()[np.argsort((acq_value_list))]
        qnehvi_x = torch.tensor(sorted_x[-12:], **tkwargs)  
        # unormalize our training inputs back to original problem bounds
        new_x =  unnormalize(qnehvi_x, bounds=problem_bounds)
        new_x = pd.DataFrame(new_x.numpy(),columns=['aspiration_rate','dispense_rate'])
        new_x['acq_value'] = sorted(acq_value_list, reverse=True)[:12]
        self._latest_suggestion = new_x[['aspiration_rate','dispense_rate']].iloc[0]
        self._latest_acq_value = new_x['acq_value'].iloc[0]
        return new_x



        ##Miscelaneous functions

    def set_data(self,df):
        iteration = 1
        nan_columns = df.columns.to_list()
        nan_columns = [e for e in nan_columns if e not in ('liquid',
        'pipette',
        'volume',
        'aspiration_rate',
        'dispense_rate',
        'blow_out_rate',
        'delay_aspirate',
        'delay_dispense',
        'delay_blow_out',
        'touch_tip_aspirate',
        'touch_tip_dispense',
        'density',
        '%error',
        'time_asp_1000',
        'acq_value',
        'iteration',
        'liquid_level')]

        df['time_asp_1000'] = 1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']

        if 'acq_value' not in df.columns:
            df['acq_value'] = None

        if df.loc[:,self.features].duplicated().sum()==0:
            df_mean = df
        else:
            df_duplicates = df.where(df.duplicated(self.features,keep=False)==True).dropna(how='all')
            df_incomplete = df.where(df.duplicated(self.features,keep=False)==False).dropna(how='all')
            df_mean = pd.DataFrame(columns= df.columns)
            for index,values in df_duplicates.drop_duplicates(self.features).iterrows():
                if len(df_duplicates.loc[index:index+2]) == len(self.mean_volumes):
                    mean_error =df_duplicates.loc[index:index+2,'%error'].abs().mean()
                    df_duplicates.loc[index,'%error'] = -mean_error
                    df_duplicates.loc[index, 'volume'] ='mean'+str(self.mean_volumes)
                    df_duplicates.loc[index, 'iteration']= iteration
                    df_duplicates.loc[index, 'liquid_level']= df.loc[index+2,'liquid_level']
                    df.loc[index:index+2, 'iteration'] = iteration
                    df_duplicates.loc[index, nan_columns]= 'NaN'
                    df_mean = pd.concat([df_mean,df.loc[index:index+2],df_duplicates.loc[[index]]])
                    iteration +=1 
                else:
                    df_incomplete = pd.concat([df_incomplete,df_duplicates.loc[index:index+2]]).drop_duplicates()
            df_mean = pd.concat([df_mean,df_incomplete])
            df_mean = df_mean.reset_index(drop=True)    
        self._data = df_mean
 

    def data_from_csv(self,file_name):
        data = pd.read_csv(file_name)
        self.set_data(data)


    def update_data(self,df):
        self._latest_volume = df['volume'].iloc[-1]
        updated_data = pd.concat([self._data,df],ignore_index=True)
        self.set_data(updated_data)
        return self._data

    
    def df_last_measurement(self,error,volume= 1000):
        self._latest_volume = volume
        last_measurement_data = self._data.iloc[[-1].copy()
        last_measurement_data.loc[updated_data.last_valid_index(),'volume'] = self._latest_volume
        last_measurement_data.loc[updated_data.last_valid_index(),'aspiration_rate']  = self._latest_suggestion['aspiration_rate']
        last_measurement_data.loc[updated_data.last_valid_index(),'dispense_rate']  = self._latest_suggestion['dispense_rate']
        last_measurement_data.loc[updated_data.last_valid_index(),'%error'] = error
        return last_measurement_data





# 3. Semi-automated implemntation

The following cells serve as an example of how to implement a semi-automated MOBO of liquid transfer paramters using the BO_LiqTransfer class. This implementation is run in parallel with the script that is controlling the liquid transfer robot. In **publication** this was used in parallel with a jupyternotbook controlling a Opentrons OT2 robot (**script link**).

The process for the MOBO is as follwos

1. Create BO_LiqTransfer object and load initial trasnfer data set. The initial transfer data set should be previously acquired through gravimmetric testing of several combination of liquid handling parameters

2. Run optimized_suggestions() method to obtain suggested liquid handling paramters by BO algorithm. This function first trains surrogate models that predict the optimization objective (default: relative error and time to aspirate 1000 µL) from predifined liquid handling parameter features (default: aspiration and dispense rates). After an acquisition function will be used to suggest new combination of liquid t ransfer paramters taht will likely minimize the objectives. 

    After, input the optimized suggestions in the script controlling the liquid handling robot and perform gravimmetric test.

3. Update the the data with each volume tested

4. Iterate steps 2 and 3 until optimal solutions are found. 


### 1. Create BO_LiqTransfer object and load initial trasnfer data set.

Please set liquid name and volume to transfer according to the experiment and load initial transfer data

In [ ]:
# Change according to experiment
liquid_name = 'Viscosity_std_1275' 

# Do not change
liq = BO_LiqTransfer(liquid_name)
liq.data_from_csv('')
liq._data


### 2.  Run optimized_suggestions() method to obtain suggested liquid handling paramters by BO algorithm.



In [ ]:
liq.optimized_suggestions()

### 3. Update the the data with each volume tested

In [ ]:
volume= 300
liq.update_data(-0.840723	,volume)


In [ ]:
#save after each standard-experiment iteration
liq._data.to_csv('C:\\Users\\amdm_\\OneDrive\\Documents\\GitHub\\viscosity_liquid_transfer_Pablo\\Opentrons_experiments\\BOTorch_optimization\\VS_code_csv\\'+liquid_name+'_'+'duplicate_unused_exp3.csv', index = False)

### 4. Iterate steps 2 and 3 until optimal solutions are found. 

## 4. Fully-automated implemetnation

The following cells serve as an example of how to implement a fully-automated MOBO of liquid transfer paramters using the BO_LiqTransfer class. This implementation is run in parallel with the script that is controlling the liquid transfer robot. In **publication** this was used to perform the multi-objective optimization of liquid handling parameters for the transport of viscous liquids with minimal human input

The process for the MOBO is as follwos

1. Initialize robotic platform
2. Create BO_LiqTransfer object by inputting liqudi name and density. 

3. Run obtainAproximateRate(). To calculate initial flow rate to be tested for the optimization protocol


4. Run exploreBoundaries(). To obtain intial data set that will be used for the optimization protocol,

5. Run optimizeParameters()

6. Run calibrateParameters()

### 1. Initialize robotic platform


For this example we use control-lab-ly python package to control an automated platform that consists of a autoamted mass balance, a Sartorious rLine1000 pipette, M1 DOBOT scara arm and a deck object cotnainig the location of the labware used in the platform.


In the initialization step a setup file defining all the automated equipment present in the platform are initialized and assigned to the *platform* variable. The *platform* variable contains the following objects:

-
-
-
-
-
-


In [4]:
#Import robot related packages and run setup
import pandas as pd
import time

from controllably.Compound import LiquidMover


pipette_settings= {
    'mover': {
        'module': 'Move', 
        'class': 'Jointed.Dobot.M1Pro', 
        'settings': {
            'ip_address': '192.109.209.21', 
            'home_coordinates': (300, 0, 200), 
            'home_orientation': (10, 0, 0), 
            'orientate_matrix': np.array([[ 0.01542615, 0.99988101, 0.],[-0.99988101, 0.01542615, 0.],[ 0., 0. , 1.]]), 
            'translate_vector': np.array([-274.51943236, 330.24797381, 283.5]), 
            'scale': 1, 'implement_offset': (0, 0, -243)
            }
        }, 
    'liquid': {
        'module': 'Transfer', 
        'class': 'Liquid.Sartorius.Sartorius',
        'settings': {'port': 'COM8', 'channel': 1, 'offset': [0, 0, 0]}
        }
    }

pipette = LiquidMover(pipette_settings)
pipette.mover.verbose = False 


    




In [ ]:
from controllably.Measure.Physical import MassBalance
balance = MassBalance(port='COM4')

In [ ]:
from controllably import load_deck
deck_layout = {
	"reference_points":{
		"1": [10.5,6.5,0],
		"2": [160.5,6.5,0],
		"3": [310.5,6.5,0],
		"4": [460.5,6.5,0]
	},
    "slots":{
		"4":{
			"name": "bin",
			"exclusion_height": -1,
			"filepath": "pe_trash.json"
		},
		"3": {
			"name": "tip_rack",
			"exclusion_height": -1,
			"filepath": "eppendorfmotion_96_tiprack_1000ul.json"
		},
		"1": {
			"name": "balance",
			"exclusion_height": -1,
			"filepath": "mass_balance_vial_40000.json"
		},
		"2": {
			"name": "reagents",
			"exclusion_height": -1,
			"filepath": "amdm_8_wellplate_30000ul.json"
		}
	}
}
load_deck(pipette, deck_layout) 

balance_deck = platform.setup.deck.slots['1'] #Objecet that holds the positional information of the balance within the cartesian space of the deck 
source = platform.setup.deck.slots['2'] #Objecet that holds the positional information of the labware containing the source of the test liquid within the cartesian space of the deck 


Since *platform* variable cotains multiple object classes, in order to make the code more readable we assign specific variables to objects that will be requiered to be used as arguments for the optimization methods in sections 3-6. 

In [ ]:
#Initialization of variables for platform objects
balance_deck = platform.setup.deck.slots['1'] #Objecet that holds the positional information of the balance within the cartesian space of the deck 
source = platform.setup.deck.slots['2'] #Objecet that holds the positional information of the labware containing the source of the test liquid within the cartesian space of the deck 

### 2. Create BO_LiqTransfer

To intizlize the BO_LiqTransfer object pass the follwing arguments:
- liquid name : String that identifies the liquid that requires liquid handling paramter optimization 
- density : Value of the density of the target liquid in g/mL, this value will be required to calculate the transfer error during the gravimmetric testing
- platform : Variable that contaisn all the automated objects of the liquid handling robot. This variable is required for the autoamted optimization of the liquid handling paramters using the methods defined in the BO_LiqTransfer() class

In [ ]:
liq = BO_LiqTransfer(liquid_name = 'Viscous_std_204',density = 0.8736, platform = platform) 

### 3. Run obtainAproximateRate()

This method contains the code that executes the actions required to obtain an approximated value of the flow rate within the pipette tip during the aspiration of the target liquid. This value will be used as an starting point for the MOBO of the liqudi handling paramters. The method required the following arguments to be pased:
- initial_liquid_level_balance : Initial height of the column of liquid in the vial is located on the balance.  
- balance_well : Object that contains the postion of the vial on the balance in the platformd deck..
- file_name : File name to save dataframe as a csv file contianing the data form the change in mass of the vial during the estimation of the approximate flow rate.


For further information refer to the protocol described in **publication**. 

In [ ]:
liq.obtainAproximateRate(initial_liquid_level_balance=7.5, balance_well= balance_deck.wells['A1'],file_name='BPAEDMA_flow_rate.csv')
liq.cleanTip(well=source.wells['A1'])

### 4. Run exploreBoundaries()

This method contains the code that executes the actions to perform the gravimetric testing of liquid handling paramters that are found at the boudnaries of the parametric space (i.e. values of maximum and minimmum aspiration rate expected). The method required the following arguments to be pased:
- initial_liquid_level_source : Initial height of the column of liquid in the vial that will be used as a source to draw for the transfer procedures.  
- source_well : Object thaht contains the postion of the vial in the platformd deck.
- balance_well : Object that contains the postion of the vial on the balance in the platformd deck.
- file_name : File name to save dataframe as a csv file contianing the data form the gravimmetric testing.


The code perfomrs the gravimetric test for the tranfer of the target volumes using the following aspiration and dispense rates:
1.  aspiration rate = Approximated flow rate , dispense rate = Approximated flow rate
2.  aspiration rate = liq.bmax x Approximated flow rate , dispense rate = liq.bmax x Approximated flow rate
3.  aspiration rate = liq.bmax x Approximated flow rate , dispense rate = liq.bmin x Approximated flow rate
4.  aspiration rate = liq.bmin x Approximated flow rate , dispense rate = liq.bmax Approximated flow rate
5.  aspiration rate = liq.bmin x Approximated flow rate , dispense rate = liq.bmin x Approximated flow rate

Where liq.max = 1.25 and liq.min = 0.1 by default.

The data gatehered during this experiments will be used to train the initial GPR for the MOBO. For further information refer to the protocol described in **publication**. 




In [ ]:
liq.exploreBoundaries(initial_liquid_level_source=42, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'])

source_iquid_level = liq._data['liquid_level'].iloc[-1]

### 5. Run optimizeParameters()
This method contains the code that executes the actions to perform the gravimetric testing of liquid handling paramters suggeted by the MOBO algorithm. The method required the following arguments to be pased:
- initial_liquid_level_source : Initial height of the column of liquid in the vial that will be used as a source to draw for the transfer procedures.  
- source_well : Object thaht contains the postion of the vial in the platformd deck.
- balance_well : Object that contains the postion of the vial on the balance in the platformd deck.
- iterations : Number of optimization iterations 
- file_name : File name to save dataframe as a csv file contianing the data form the gravimmetric testing.



In [ ]:
liq.optimize_parameters(initial_liquid_level_source = source_iquid_level, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'], iterations=5, file_name = 'Viscosity_std_204.csv')


### 6. Run calibrateParameters()

In [ ]:
liq.calibrate_parameters(46.5,source_well=source.wells['A1'],balance_well=balance_deck.wells['A1'],file_name='Viscosity_std_204_calibration.csv')